# A. Setting up Google Colab and Drive

In [ ]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")

In [ ]:
!ls

MS-SNSD  noise_suppressor.ipynb


# B. Setting up the required Data

### Cloning Dataset Repo

!git clone https://github.com/microsoft/MS-SNSD.git

### Installing required modules for creating required data

In [10]:
!pip install -r ./MS-SNSD/requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: './MS-SNSD/requirements.txt'


In [2]:
!pip install pysoundfile

#### As of the date when this code was developed, Dataset processing works only with the below NumPy version

In [3]:
!python -m pip install numpy==1.16.4

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1023, in _handle_fromlist
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 83, in <module>
    __import__('pip._vendor.packaging.specifiers')
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/packaging/specifiers.py", line 266, in <module>
    class Specifier(_IndividualSpecifier):
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/packaging/specifiers.py", line 361, in Specifier
    _regex = re.compile(r"^\s*" + _regex_str + r"\s*$", re.VERBOSE | re.IGNORECASE)
  File "/usr/lib/python3.6/re.py", line 233, in compile
    return _compile(pattern, flags)
  File "/usr/lib/python3.6/re.py", line 301, in _compile
    p = sre_compile.compile(pattern, flags)
  File "/usr/lib/python3.6/sre_compile.py", line 566, in compile
    code = _code(p, flags)
  File "/usr/lib/python3.6/sr

In [4]:
!rm -rf /content/MS-SNSD/CleanSpeech_training

In [5]:
!rm -rf /content/MS-SNSD/NoisySpeech_training

In [6]:
!pwd

/content


In [7]:
pip list | grep librosa

librosa                       0.6.3          


### Run the python file in the dataset to generated the noisy_voice and clean_voice data

In [8]:
!python ./MS-SNSD/noisyspeech_synthesizer.py

python3: can't open file './MS-SNSD/noisyspeech_synthesizer.py': [Errno 2] No such file or directory


In [9]:
!pip install librosa

# C. Building the noise suppressor model

### 1. Library imports

In [11]:
import librosa
import math
import numpy as np
import os
from scipy import stats

import matplotlib.pyplot as plt
import matplotlib as mpl
import librosa.display

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, MaxPooling2D, Dropout, concatenate, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend
import tensorflow as tf

### 2. Declaring Constant variables

In [ ]:
DIM_SQ_SPEC = 128  # dimension required for the UNet to process the spectrogram input
SAMPLE_RATE = 8000
N_FFT = DIM_SQ_SPEC*2-1
HOP_LENGTH = N_FFT//4 
SQ_CLIP_LIMIT = math.floor(N_FFT/4)*math.ceil(N_FFT/2)

print("N_FFT:", N_FFT)
print("HOP_LENGTH:", HOP_LENGTH)
print("SQ_CLIP_LIMIT:", SQ_CLIP_LIMIT)

N_FFT: 255
HOP_LENGTH: 63
SQ_CLIP_LIMIT: 8064


### 3. Data preprocessing

#### 3a. Split the audio signal into smaller frames to ensure a Square Spectrogram can be created from it.

In [ ]:
NOISE_DATASET_PATH = "/content/drive/My Drive/Colab Notebooks/MS-SNSD/NoisySpeech_training"
CLEAN_DATASET_PATH = "/content/drive/My Drive/Colab Notebooks/MS-SNSD/CleanSpeech_training"

In [ ]:
def fetch_filenames(path):
    for (dirpath, dirnames, filenames) in os.walk(path):
        return filenames

In [ ]:
def signal_creator(noise_path, clean_path):
    noise_signal, noise_sr = librosa.load(noise_path, sr=SAMPLE_RATE)
    clean_signal, clean_sr = librosa.load(clean_path, sr=SAMPLE_RATE)
    only_noise = noise_signal[:] - clean_signal[:]
    return noise_signal, clean_signal, only_noise

In [ ]:
def numpy_frame_creator(signal, signal_length=int(SAMPLE_RATE*1.1)):
    frame_signal = librosa.util.frame(signal[:signal_length], frame_length=SQ_CLIP_LIMIT, hop_length=SQ_CLIP_LIMIT).T
    return frame_signal

In [ ]:
SAMPLE_RATE*1.1

8800.0

In [ ]:
''' convert all .wav files to smaller frames to create spectograms with amplitude and phase component. '''
# fetch and store the audio numpy array
noise_filenames = fetch_filenames(NOISE_DATASET_PATH)
clean_filenames = fetch_filenames(CLEAN_DATASET_PATH)

print("Number of files: ", len(noise_filenames), len(clean_filenames))

noise_frames = []
clean_frames = []
only_noise_frames = []
for i, noise_file in enumerate(noise_filenames):
    clean_file = noise_file.split("_")[-1]
    if (clean_file in clean_filenames):
        noise_file = os.path.join(NOISE_DATASET_PATH, noise_file)
        clean_file = os.path.join(CLEAN_DATASET_PATH, clean_file)
        noise_signal, clean_signal, only_noise_signal = signal_creator(noise_file, clean_file)
        noise_set = numpy_frame_creator(noise_signal)
        clean_set = numpy_frame_creator(clean_signal)
        only_noise_set = numpy_frame_creator(only_noise_signal)
        noise_frames.append(noise_set)
        clean_frames.append(clean_set)
        only_noise_frames.append(only_noise_set)
noise_frames = np.vstack(noise_frames)
clean_frames = np.vstack(clean_frames)
only_noise_frames = np.vstack(only_noise_frames)


print("Shape of noise clean and only noise frames: ", noise_frames.shape, clean_frames.shape, only_noise_frames.shape)

# divide them into frames and vstack and store them.
# compute stft for them and store the mag and phase as seperate numpy arrays.

In [ ]:
np.save("/content/drive/My Drive/Colab Notebooks/signal_arr/noise_signal_8khz_1s", noise_frames)
np.save("/content/drive/My Drive/Colab Notebooks/signal_arr/clean_signal_8khz_1s", clean_frames)
np.save("/content/drive/My Drive/Colab Notebooks/signal_arr/only_noise_signal_8khz_1s", only_noise_frames)

#### 3b. Creating the spectogram for the frames

In [ ]:
def specto_mag_phase(signal_frame):
    stft = librosa.core.stft(signal_frame, n_fft=N_FFT, hop_length=HOP_LENGTH)
    stft_mag, stft_ph = librosa.core.magphase(stft)
    stft_mag_db = librosa.core.amplitude_to_db(stft_mag)
    return stft_mag_db, stft_ph

In [ ]:
def signal_to_spect(signal):
    n_frames = signal.shape[0]
    signal_mag = np.zeros((n_frames, DIM_SQ_SPEC, DIM_SQ_SPEC))
    signal_ph = np.zeros((n_frames, DIM_SQ_SPEC, DIM_SQ_SPEC), dtype=complex)

    for i in range(n_frames):
        signal_mag[i, :, :], signal_ph[i, :, :] = specto_mag_phase(signal[i])
    return signal_mag, signal_ph

In [ ]:
noise_frames = np.load("/content/drive/My Drive/Colab Notebooks/signal_arr/noise_signal_8khz_1s.npy")
noise_stft_mag, noise_stft_ph = signal_to_spect(noise_frames)
print("Noise STFT Mag and Phase shape: ", noise_stft_mag.shape, noise_stft_ph.shape)

np.save("/content/drive/My Drive/Colab Notebooks/stft_mag_arr/noise_stft_mag", noise_stft_mag)

Noise STFT Mag and Phase shape:  (1815, 128, 128) (1815, 128, 128)


In [ ]:
clean_frames = np.load("/content/drive/My Drive/Colab Notebooks/signal_arr/clean_signal_8khz_1s.npy")
clean_stft_mag, clean_stft_ph = signal_to_spect(clean_frames)
print("Clean STFT Mag and Phase shape: ", clean_stft_mag.shape, clean_stft_ph.shape)

np.save("/content/drive/My Drive/Colab Notebooks/stft_mag_arr/clean_stft_mag", clean_stft_mag)

Clean STFT Mag and Phase shape:  (1815, 128, 128) (1815, 128, 128)


In [ ]:
only_noise_frames = np.load("/content/drive/My Drive/Colab Notebooks/signal_arr/only_noise_signal_8khz_1s.npy")
only_noise_stft_mag, only_noise_stft_ph = signal_to_spect(only_noise_frames)
print("Only Noise STFT Mag and Phase shape: ", only_noise_stft_mag.shape, only_noise_stft_ph.shape)

np.save("/content/drive/My Drive/Colab Notebooks/stft_mag_arr/only_noise_stft_mag", only_noise_stft_mag)

Only Noise STFT Mag and Phase shape:  (1815, 128, 128) (1815, 128, 128)


### 4. Normalize the STFT components between 0 and 1

In [ ]:
inp_x = np.load("/content/drive/My Drive/Colab Notebooks/stft_mag_arr/noise_stft_mag.npy")
out_x = np.load("/content/drive/My Drive/Colab Notebooks/stft_mag_arr/only_noise_stft_mag.npy")

In [ ]:
print(stats.describe(inp_x.reshape(-1,1)))
print(stats.describe(out_x.reshape(-1,1)))

DescribeResult(nobs=29736960, minmax=(array([-81.46199036]), array([31.83718109])), mean=array([-30.55498533]), variance=array([288.22655219]), skewness=array([0.11872211]), kurtosis=array([-0.49748866]))
DescribeResult(nobs=29736960, minmax=(array([-100.]), array([31.83179092])), mean=array([-38.5884761]), variance=array([358.96261668]), skewness=array([-0.3461536]), kurtosis=array([-0.00910125]))


In [ ]:
out_x.shape

(1815, 128, 128)

In [ ]:
def scale_in(specto):
    specto = (specto + 82)/114
    return specto

In [ ]:
def scale_out(specto):
    specto = (specto + 100 )/132
    return specto

In [ ]:
norm_inp_x = scale_in(inp_x)
norm_out_x = scale_out(out_x)

norm_inp_x = norm_inp_x[..., np.newaxis]
norm_out_x = norm_out_x[..., np.newaxis]

print("Shape of inp arr: ", norm_inp_x.shape)
print("Shape of out arr: ", norm_out_x.shape)

Shape of inp arr:  (1815, 128, 128, 1)
Shape of out arr:  (1815, 128, 128, 1)


In [ ]:

print(stats.describe(norm_inp_x.reshape(-1,1)))
print(stats.describe(norm_out_x.reshape(-1,1)))

DescribeResult(nobs=29736960, minmax=(array([0.00471938]), array([0.99857176])), mean=array([0.45127206]), variance=array([0.0221781]), skewness=array([0.11872211]), kurtosis=array([-0.49748866]))
DescribeResult(nobs=29736960, minmax=(array([0.]), array([0.99872569])), mean=array([0.46523882]), variance=array([0.02060162]), skewness=array([-0.3461536]), kurtosis=array([-0.00910125]))


### 5. Define the UNET model architecture.

In [ ]:
def unet(pretrained_weights = None,input_size = (128,128,1)):
    #size filter input
    size_filter_in = 16
    #normal initialization of weights
    kernel_init = 'he_normal'
    #To apply leaky relu after the conv layer 
    activation_layer = None
    inputs = Input(input_size)
    conv1 = Conv2D(size_filter_in, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(inputs)
    conv1 = LeakyReLU()(conv1)
    conv1 = Conv2D(size_filter_in, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv1)
    conv1 = LeakyReLU()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(size_filter_in*2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(pool1)
    conv2 = LeakyReLU()(conv2)
    conv2 = Conv2D(size_filter_in*2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv2)
    conv2 = LeakyReLU()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(size_filter_in*4, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(pool2)
    conv3 = LeakyReLU()(conv3)
    conv3 = Conv2D(size_filter_in*4, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv3)
    conv3 = LeakyReLU()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(size_filter_in*8, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(pool3)
    conv4 = LeakyReLU()(conv4)
    conv4 = Conv2D(size_filter_in*8, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv4)
    conv4 = LeakyReLU()(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(size_filter_in*16, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(pool4)
    conv5 = LeakyReLU()(conv5)
    conv5 = Conv2D(size_filter_in*16, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv5)
    conv5 = LeakyReLU()(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(size_filter_in*8, 2, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(UpSampling2D(size = (2,2))(drop5))
    up6 = LeakyReLU()(up6)
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(size_filter_in*8, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(merge6)
    conv6 = LeakyReLU()(conv6)
    conv6 = Conv2D(size_filter_in*8, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv6)
    conv6 = LeakyReLU()(conv6)
    up7 = Conv2D(size_filter_in*4, 2, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(UpSampling2D(size = (2,2))(conv6))
    up7 = LeakyReLU()(up7)
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(size_filter_in*4, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(merge7)
    conv7 = LeakyReLU()(conv7)
    conv7 = Conv2D(size_filter_in*4, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv7)
    conv7 = LeakyReLU()(conv7)
    up8 = Conv2D(size_filter_in*2, 2, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(UpSampling2D(size = (2,2))(conv7))
    up8 = LeakyReLU()(up8)
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(size_filter_in*2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(merge8)
    conv8 = LeakyReLU()(conv8)
    conv8 = Conv2D(size_filter_in*2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv8)
    conv8 = LeakyReLU()(conv8)

    up9 = Conv2D(size_filter_in, 2, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(UpSampling2D(size = (2,2))(conv8))
    up9 = LeakyReLU()(up9)
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(size_filter_in, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(merge9)
    conv9 = LeakyReLU()(conv9)
    conv9 = Conv2D(size_filter_in, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv9)
    conv9 = LeakyReLU()(conv9)
    conv9 = Conv2D(2, 3, activation = activation_layer, padding = 'same', kernel_initializer = kernel_init)(conv9)
    conv9 = LeakyReLU()(conv9)
    conv10 = Conv2D(1, 1, activation = 'tanh')(conv9)

    model = Model(inputs,conv10)

    model.compile(optimizer = 'adam', loss = 'huber', metrics = ['mae'])

    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model


### 6. Create the Train-Test split, initialise the model and checkpoints

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(norm_inp_x, norm_out_x, test_size=0.1)

model = unet()
chkpoint = ModelCheckpoint("/content/drive/My Drive/Colab Notebooks/model_chkpoint/best_model.h5", verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
model.summary()

Model: "functional_45"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_528 (Conv2D)             (None, 128, 128, 16) 160         input_23[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_506 (LeakyReLU)     (None, 128, 128, 16) 0           conv2d_528[0][0]                 
__________________________________________________________________________________________________
conv2d_529 (Conv2D)             (None, 128, 128, 16) 2320        leaky_re_lu_506[0][0]            
______________________________________________________________________________________

In [ ]:
del norm_inp_x, norm_out_x, inp_x, out_x

### 7. Train the model

In [ ]:
model.fit(x_train, y_train, epochs=20, batch_size=20, shuffle=True, callbacks=[chkpoint], verbose=1, validation_data=(x_test, y_test))

Epoch 1/20
82/82 [==============================] - ETA: 0s - loss: 0.0069 - mae: 0.0814
Epoch 00001: val_loss did not improve from 0.00334
82/82 [==============================] - 13s 163ms/step - loss: 0.0069 - mae: 0.0814 - val_loss: 0.0156 - val_mae: 0.1473
Epoch 2/20
82/82 [==============================] - ETA: 0s - loss: 0.0031 - mae: 0.0554
Epoch 00002: val_loss did not improve from 0.00334
82/82 [==============================] - 13s 157ms/step - loss: 0.0031 - mae: 0.0554 - val_loss: 0.0042 - val_mae: 0.0603
Epoch 3/20
82/82 [==============================] - ETA: 0s - loss: 0.0025 - mae: 0.0497
Epoch 00003: val_loss did not improve from 0.00334
82/82 [==============================] - 13s 156ms/step - loss: 0.0025 - mae: 0.0497 - val_loss: 0.0042 - val_mae: 0.0598
Epoch 4/20
82/82 [==============================] - ETA: 0s - loss: 0.0025 - mae: 0.0495
Epoch 00004: val_loss did not improve from 0.00334
82/82 [==============================] - 13s 157ms/step - loss: 0.0025 - m

##### Model metrics: loss: 0.0011 - mae: 0.0312 - val_loss: 0.0030 - val_mae: 0.0523 at Epoch 20

In [ ]:
model.save("saved_model/mag_best_model.h5")

### 8. Model Evaluation and Testing on samples

In [ ]:
def inv_scaled_out(specto):
    specto = specto * 132 - 100
    return specto

In [ ]:
def mag_ph_to_audio(mag, ph):
    mag = librosa.core.db_to_amplitude(mag)
    stft = mag*ph
    istft = librosa.core.istft(stft)
    return istft

In [ ]:
def specto_to_signal(specto, ph):
    audio_signal = []
    n_frames = specto.shape[0]
    for i in range(n_frames):
        audio_recons = mag_ph_to_audio(specto[i], ph[i])
        audio_signal.append(audio_recons)
    
    return np.vstack(audio_signal)

In [ ]:
loaded_model = tf.keras.models.load_model('saved_model/mag_best_model.h5')
pred_audio_file = "/content/drive/My Drive/Colab Notebooks/test voice.aac"  

signal, c_sig, o_n_sign = signal_creator(pred_audio_file, pred_audio_file)
pred_frames = numpy_frame_creator(signal=signal, signal_length=len(signal)) 
print("Shape of raw signal: ")

sp_mag, sp_ph = signal_to_spect(pred_frames) 

print("Spec mag shape: ", sp_mag.shape)
print("Spec ph shape: ", sp_ph.shape)

sp_mag_1 = scale_in(sp_mag) 
pred_x = sp_mag_1[..., np.newaxis]
ph_pred_x = sp_ph[..., np.newaxis]

print("Shape of input pred: ", pred_x.shape)


pred_out = loaded_model.predict(pred_x)
inv_scaled_pred_out = inv_scaled_out(pred_out) 
inv_scaled_pred_out = inv_scaled_pred_out.reshape(inv_scaled_pred_out.shape[0], inv_scaled_pred_out.shape[1], inv_scaled_pred_out.shape[2])


noise_mask =  inv_scaled_pred_out   
print("Shape of noise spec: ", noise_mask.shape)


noise_mask_recons_numpy = specto_to_signal(noise_mask, sp_ph)
print("Shape of noise signal: ", noise_mask_recons_numpy.shape)

nb_samples = noise_mask_recons_numpy.shape[0]
noise_mask_signal = (noise_mask_recons_numpy.reshape(-1,).T)
print("Shape of noise mask: ", noise_mask_signal.shape)
librosa.output.write_wav("test_voice_predicted_noise_mask.wav", noise_mask_signal, SAMPLE_RATE)





input_mag = sp_mag     
print("Shape of input spec: ", input_mag.shape)


input_audio_numpy = specto_to_signal(input_mag, sp_ph)
print("Shape of input signal: ", input_audio_numpy.shape)

nb_samples = input_audio_numpy.shape[0]
input_audio_signal = (input_audio_numpy.reshape(-1,).T)
print("Shape of input signal: ", input_audio_signal.shape)
librosa.output.write_wav("test_voice_original_clip.wav", input_audio_signal, SAMPLE_RATE)



librosa.output.write_wav("test_voice_denoised_input.wav", input_audio_signal-noise_mask_signal, SAMPLE_RATE)

### 9. Graph plots of spectograms

In [ ]:



def make_plot_spectrogram(stftaudio_magnitude_db,sample_rate, hop_length_fft) :
    """This function plots a spectrogram"""
    plt.figure(figsize=(12, 6))
    librosa.display.specshow(stftaudio_magnitude_db, x_axis='time', y_axis='linear',
                             sr=sample_rate, hop_length=hop_length_fft)
    plt.colorbar()
    title = 'hop_length={},  time_steps={},  fft_bins={}  (2D resulting shape: {})'
    plt.title(title.format(hop_length_fft,
                           stftaudio_magnitude_db.shape[1],
                           stftaudio_magnitude_db.shape[0],
                           stftaudio_magnitude_db.shape));
    return

def make_plot_phase(stft_phase,sample_rate,hop_length_fft) :
    """This function plots the phase in radian"""
    plt.figure(figsize=(12, 6))
    librosa.display.specshow(np.angle(stft_phase), x_axis='time', y_axis='linear',
                             sr=sample_rate, hop_length=hop_length_fft)
    plt.colorbar()
    title = 'hop_length={},  time_steps={},  fft_bins={}  (2D resulting shape: {})'
    plt.title(title.format(hop_length_fft,
                           stft_phase.shape[1],
                           stft_phase.shape[0],
                           stft_phase.shape));
    return

def make_plot_time_serie(audio,sample_rate):
    """This function plots the audio as a time serie"""
    plt.figure(figsize=(12, 6))
    #plt.ylim(-0.05, 0.05)
    plt.title('Audio')
    plt.ylabel('Amplitude')
    plt.xlabel('Time(s)')
    librosa.display.waveplot(audio, sr=sample_rate)
    return


def make_3plots_spec_voice_noise(stftvoicenoise_mag_db,stftnoise_mag_db,stftvoice_mag_db,sample_rate, hop_length_fft):
    """This function plots the spectrograms of noisy voice, noise and voice as a single plot """
    plt.figure(figsize=(8, 12))
    plt.subplot(3, 1, 1)
    plt.title('Spectrogram voice + noise')
    librosa.display.specshow(stftvoicenoise_mag_db, x_axis='time', y_axis='linear',sr=sample_rate, hop_length=hop_length_fft)
    plt.colorbar()
    plt.subplot(3, 1, 2)
    plt.title('Spectrogram predicted voice')
    librosa.display.specshow(stftnoise_mag_db, x_axis='time', y_axis='linear',sr=sample_rate, hop_length=hop_length_fft)
    plt.colorbar()
    plt.subplot(3, 1, 3)
    plt.title('Spectrogram true voice')
    librosa.display.specshow(stftvoice_mag_db, x_axis='time', y_axis='linear',sr=sample_rate, hop_length=hop_length_fft)
    plt.colorbar()
    plt.tight_layout()

    return


def make_3plots_phase_voice_noise(stftvoicenoise_phase,stftnoise_phase,stftvoice_phase,sample_rate, hop_length_fft):
    """This function plots the phase in radians of noisy voice, noise and voice as a single plot """
    plt.figure(figsize=(8, 12))
    plt.subplot(3, 1, 1)
    plt.title('Phase (radian) voice + noise')
    librosa.display.specshow(np.angle(stftvoicenoise_phase), x_axis='time', y_axis='linear',sr=sample_rate, hop_length=hop_length_fft)
    plt.colorbar()
    plt.subplot(3, 1, 2)
    plt.title('Phase (radian) predicted voice')
    librosa.display.specshow(np.angle(stftnoise_phase), x_axis='time', y_axis='linear',sr=sample_rate, hop_length=hop_length_fft)
    plt.colorbar()
    plt.subplot(3, 1, 3)
    plt.title('Phase (radian) true voice')
    librosa.display.specshow(np.angle(stftvoice_phase), x_axis='time', y_axis='linear',sr=sample_rate, hop_length=hop_length_fft)
    plt.colorbar()
    plt.tight_layout()

    return


def make_3plots_timeseries_voice_noise(clipvoicenoise,clipnoise,clipvoice, sample_rate) :
    """This function plots the time series of audio of noisy voice, noise and voice as a single plot """
    #y_ax_min = min(clipvoicenoise) - 0.15
    #y_ax_max = max(clipvoicenoise) + 0.15

    plt.figure(figsize=(18, 12))
    plt.subplots_adjust(hspace=0.35)
    plt.subplot(3, 1, 1)
    plt.title('Audio voice + noise')
    plt.ylabel('Amplitude')
    plt.xlabel('Time(s)')
    librosa.display.waveplot(clipvoicenoise, sr=sample_rate)
    plt.ylim(-0.05, 0.05)
    plt.subplot(3, 1, 2)
    plt.title('Audio predicted voice')
    plt.ylabel('Amplitude')
    plt.xlabel('Time(s)')
    librosa.display.waveplot(clipnoise, sr=sample_rate)
    plt.ylim(-0.05, 0.05)
    plt.subplot(3, 1, 3)
    plt.title('Audio true voice')
    plt.ylabel('Amplitude')
    plt.xlabel('Time(s)')
    librosa.display.waveplot(clipvoice, sr=sample_rate)
    plt.ylim(-0.05, 0.05)

    return

# for i in [8, 15, 20, 21, 26, 33]:
file_number = "33" # "my_voice"

input_file = "eval_input_%s.wav"%file_number
output_file = "eval_output_%s.wav"%file_number
actual_clean_file = "clnsp%s.wav"%file_number

input_plot_name = input_file.split("_")[-1].split(".")[0]
output_plot_name = output_file.split("_")[-1].split(".")[0]
actual_clean_plot_name = actual_clean_file.split(".")[0][-2:]

plt.figure(figsize=(8, 12))
plt.subplots_adjust(hspace=0.35)
plt.subplot(3, 1, 1)
plt.title("Noisy Voice Input Evaluation Sample (%s)"%(file_number))
plt.xlabel("Time")
plt.ylabel("Frequency")
input_signal, input_sr = librosa.load(input_file, sr=SAMPLE_RATE)
print(len(input_signal))
input_stft = librosa.stft(input_signal, n_fft=N_FFT)
input_magnitude, input_phase = librosa.magphase(input_stft)
in_log_spec = librosa.amplitude_to_db(input_magnitude)
librosa.display.specshow(in_log_spec, sr=SAMPLE_RATE, hop_length=HOP_LENGTH, cmap="inferno")
plt.colorbar()
#plt.savefig("input_%s"%(file_number))
#plt.show()

plt.subplot(3, 1, 2)
plt.title("Denoised Voice Output Evaluation Sample (%s)"%(file_number))
plt.xlabel("Time")
plt.ylabel("Frequency")
output_signal, output_sr = librosa.load(output_file, sr=SAMPLE_RATE)
print(len(output_signal))
output_stft = librosa.stft(output_signal, n_fft=N_FFT)
output_magnitude, output_phase = librosa.magphase(output_stft)
out_log_spec = librosa.amplitude_to_db(output_magnitude)
librosa.display.specshow(out_log_spec, sr=SAMPLE_RATE, hop_length=HOP_LENGTH, cmap="inferno")
plt.colorbar()
#plt.savefig("output_%s"%(file_number))
#plt.show()

plt.subplot(3, 1, 3)
plt.title("Actual Clean voice of Evaluation Sample (%s)"%(actual_clean_file))
plt.xlabel("Time")
plt.ylabel("Frequency")
actual_signal, actual_sr = librosa.load(actual_clean_file, sr=SAMPLE_RATE)
print(len(actual_signal[:len(output_signal)]))
actual_stft = librosa.stft(actual_signal[:len(output_signal)], n_fft=N_FFT)
actual_magnitude, actual_phase = librosa.magphase(actual_stft)
actual_log_spec = librosa.amplitude_to_db(actual_magnitude)
librosa.display.specshow(actual_log_spec, sr=SAMPLE_RATE, hop_length=HOP_LENGTH, cmap="inferno")
plt.colorbar()
plt.savefig("evaluation_sample_all_three_%s"%(file_number))
#plt.show()